In [23]:
import os

In [24]:
#%pwd

In [25]:
#os.chdir("../")

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str



In [27]:
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories



In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        return create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [29]:
import pandas as pd
import os
from sklearn.linear_model import ElasticNet
from src.project.utils import logger
import joblib

In [32]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        x_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]

        x_test = test_data.drop([self.config.target_column], axis = 1)
        y_test = test_data[[self.config.target_column]]

        elastic_model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state = 42)
        elastic_model.fit(x_train,y_train)

        joblib.dump(elastic_model, os.path.join(self.config.root_dir, self.config.model_name))




In [33]:
try:
    configuarion_manager = ConfigurationManager()
    model_trainer_config = configuarion_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-09-15 07:48:41,158: INFO: common: created directory at: artifacts]
[2025-09-15 07:48:41,160: INFO: common: created directory at: artifacts/model_trainer]
